In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'AERO_402_Further_Assessment'

stk_object = STK_Simulation(False,Filename)

In [4]:
# Setting scenario time variables
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")
n_pop = 10
n_gen = 10
n_sats = 7

opt = Optimizer(stk_object,n_pop,n_gen,n_sats,weights=(7,-5,-1))

c:\Users\rober\Python\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.

c:\Users\rober\Python\Lib\site-packages\deap\creator.py:185: RuntimeWarning:

A class named 'Satellite' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.



In [3]:
# Loading targets into stk from file.
stk_object.Target_Loader("../../Input_Files/Targets_34.txt")

In [5]:
stk_object.dt = 30
stk_object.Interpolate=True

# Run optimization if read=True the 1st generation takes the input file as one of the initial population
hof,percent,std,time = opt.run(read=True)

print("Hall of Fame:")
for entry in hof:
    print(entry,"->",entry.fitness)

# Get best individual from optimization and load them into ../../Input_Files/Satellite_File.txt

best_stats = opt.cost_function(hof[0])

-- Generation 2 --
         avg        std        min        max
0  62.487291  18.154992  29.856572  79.892883
1  11.436238   7.778527   5.628389  34.120993
2  23.400682   0.402201  22.936654  24.000000


In [9]:

stk_object.dt = 2.5
stk_object.Generate_Holding_Data()

- Computing_AER Data/Default [          ] ▁▃▅ 0/75 [0%] in 0s (~0s, 0.0/s) 

- Computing_AER Data/Default [==========] 75/75 [100%] in 2:08.3 (0.58/s)       


on 9: Exception ignored in: <function LPSAFEARRAY_arg.__del__ at 0x0000020326927600>


on 9: Traceback (most recent call last):
  File "c:\Users\rober\Python\Lib\site-packages\agi\stk12\internal\marshall.py", line 922, in __del__
    self._cleanup()


on 9:   File "c:\Users\rober\Python\Lib\site-packages\agi\stk12\internal\marshall.py", line 920, in _cleanup
    agcom.oleaut32lib.SafeArrayDestroy(self.sa)


on 9: KeyboardInterrupt:


- Computing_Sat Angles Data [==>       ] ▆▄▂ 20/75 [27%] in 22s (~1:00, 0.9/s)  

In [6]:
stk_object.Plan(slew_rate=1,cone_angle=20)

- Holding_Data [==========] 4860/4860 [100%] in 18.1s (271.61/s)                [==========] ▅▇▇ 4837/4860 [100%] in 17s (~0s, 293.1/s) 


0

In [7]:
data_comparison = {}
data_comparison["Unplanned (%)"] = [len(np.unique(stk_object.Holding_Data[stk_object.Holding_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (%)"] = [len(np.unique(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Bin Number'].values))/324*100 for tar_num in range(len(stk_object.targets))]
data_comparison["Planned (Time)"] = [max(stk_object.Planned_Data[stk_object.Planned_Data['Target'].values==tar_num]['Time'].values)/86400 for tar_num in range(len(stk_object.targets))]

data_comparison = pd.DataFrame(data_comparison)
print(data_comparison)
print(data_comparison.describe())

    Unplanned (%)  Planned (%)  Planned (Time)
0       74.691358    20.061728        0.999120
1       72.530864    23.456790        0.993206
2       72.222222    17.283951        0.990116
3       70.679012    14.506173        0.994178
4       71.296296    16.666667        0.995446
5       71.296296    16.049383        0.993716
6       68.518519    17.901235        0.991198
7       69.135802    13.888889        0.927569
8       65.123457    18.518519        0.988669
9       72.839506    21.913580        0.987743
10      69.753086    15.123457        0.985948
11      69.753086    17.901235        0.986476
12      73.456790    23.148148        0.920770
13      69.135802    18.827160        0.987685
14      64.814815    13.888889        0.997471
       Unplanned (%)  Planned (%)  Planned (Time)
count      15.000000    15.000000       15.000000
mean       70.349794    17.942387        0.982621
std         2.800871     3.129911        0.024093
min        64.814815    13.888889        0.92077

In [8]:
fig = px.scatter(stk_object.Planned_Data.sort_values(by="Target"),x="Time",y="Bin Number",hover_data="Bin Number",color="Satellite",animation_frame="Target")
fig.show()

In [44]:
df2 = pd.DataFrame({
    'Unplanned (%)': [72.530864, 74.074074, 78.703704, 75.000000, 63.888889, 74.074074, 76.234568, 62.345679, 72.222222, 72.839506],
    'Planned (%)': [49.074074, 48.765432, 33.024691, 40.123457, 28.086420, 44.135802, 32.716049, 22.530864, 44.135802, 36.419753],
    'Planned (Time)': [0.956678, 0.958640, 0.962818, 0.957703, 0.953704, 0.961441, 0.887737, 0.949340, 0.962025, 0.960208]
})

In [28]:
import pandas as pd
import openpyxl

In [56]:

dfs_to_excel(r"H:/Shared drives/AERO 401 Project  L3Harris Team 1/Subteam Designs/OCD/Superstars.xlsx", str(n_sats), df1=pd.read_csv("../../Input_Files/Satellites_File.txt"), df2=data_comparison)